In [1]:
from sentence_transformers.cross_encoder import CrossEncoder
import torch

# MODEL_NAME = "hotchpotch/tmp-mb-jp-30m-reranker"
# MODEL_NAME = "../outputs/exp336"
# MODEL_NAME = "cl-nagoya/ruri-v3-reranker-310m"
MODEL_NAME = "../outputs/exp337"
# MODEL_NAME = "BAAI/bge-reranker-v2-m3"
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CrossEncoder(MODEL_NAME, device=device, max_length=512)


/home/hotchpotch/src/github.com/hotchpotch/yart/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [2]:
model.predict([["日本の首都はどこですか？", "東京"]], show_progress_bar=True)

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.38it/s]


array([0.00290549], dtype=float32)

In [3]:
# query = "教育向けの日本語データセットの公開"
query = "感動的な映画について"

passages = [
    "深いテーマを持ちながらも、観る人の心を揺さぶる名作。登場人物の心情描写が秀逸で、ラストは涙なしでは見られない。",
    "重要なメッセージ性は評価できるが、暗い話が続くので気分が落ち込んでしまった。もう少し明るい要素があればよかった。",
    "どうにもリアリティに欠ける展開が気になった。もっと深みのある人間ドラマが見たかった。",
    "アクションシーンが楽しすぎる。見ていて飽きない。ストーリーはシンプルだが、それが逆に良い。",
    "2024年12月に公開された FineWeb2 は多言語対応（日本語を含む）の高品質データセットですが、2025年2月現在、教育向けに価値が高い「Edu」データセットは未公開です。そこで、本プロジェクトでは FineWeb2 Edu Japanese データセット を作成し、公開しました。",
    "miracl データセットはマルチリンガルの研究者向けデータセットです。",
]
scores = model.rank(query, passages)


In [4]:
for data in scores:
    score = data["score"]
    corpus_id = data["corpus_id"]
    passage = passages[corpus_id]
    print(f"Score: {score:.4f}, Passage: {passage}")

Score: 0.5413, Passage: 2024年12月に公開された FineWeb2 は多言語対応（日本語を含む）の高品質データセットですが、2025年2月現在、教育向けに価値が高い「Edu」データセットは未公開です。そこで、本プロジェクトでは FineWeb2 Edu Japanese データセット を作成し、公開しました。
Score: 0.1228, Passage: 深いテーマを持ちながらも、観る人の心を揺さぶる名作。登場人物の心情描写が秀逸で、ラストは涙なしでは見られない。
Score: 0.0579, Passage: アクションシーンが楽しすぎる。見ていて飽きない。ストーリーはシンプルだが、それが逆に良い。
Score: 0.0445, Passage: どうにもリアリティに欠ける展開が気になった。もっと深みのある人間ドラマが見たかった。
Score: 0.0257, Passage: 重要なメッセージ性は評価できるが、暗い話が続くので気分が落ち込んでしまった。もう少し明るい要素があればよかった。
Score: 0.0127, Passage: miracl データセットはマルチリンガルの研究者向けデータセットです。


In [5]:
import datasets

ds_test = datasets.load_dataset("hotchpotch/japanese-query-craft", split="test")

In [6]:
queries = ds_test["query"]
passages = ds_test["text"]
# zip
query_passages = list(zip(queries, passages))
scores = model.predict(query_passages, show_progress_bar=True)

Batches: 100%|██████████| 24/24 [00:01<00:00, 18.03it/s]


In [7]:
# np で統計表示
import numpy as np


def print_statistics(scores):
    print(f"Mean: {np.mean(scores):.4f}")
    print(f"Median: {np.median(scores):.4f}")
    print(f"Max: {np.max(scores):.4f}")
    print(f"Min: {np.min(scores):.4f}")
    print(f"Std: {np.std(scores):.4f}")


print_statistics(scores)

Mean: 0.6851
Median: 0.9119
Max: 0.9998
Min: 0.0000
Std: 0.3704


In [8]:
# 10% 刻みのパーセンタイルを表示
def print_percentiles(scores):
    percentiles = [np.percentile(scores, p) for p in range(0, 101, 10)]
    for p, score in zip(range(0, 101, 10), percentiles):
        print(f"{p:3d} %: {score:.4f}")


print_percentiles(scores)


  0 %: 0.0000
 10 %: 0.0643
 20 %: 0.1827
 30 %: 0.5032
 40 %: 0.7538
 50 %: 0.9119
 60 %: 0.9620
 70 %: 0.9831
 80 %: 0.9928
 90 %: 0.9971
100 %: 0.9998


In [9]:
# scoreが0.2以下のデータを表示
def print_low_scores(scores, query_passages):
    low_scores = [
        (query, passage, score)
        for (query, passage), score in zip(query_passages, scores)
        if score <= 0.1
    ]
    for query, passage, score in low_scores:
        short_passage = passage  # .replace("\n", " ")
        print(f"{score:.4f}: {query}")
        print(f"{short_passage}")
        print("-------------------------------------")
    return len(low_scores)


print_low_scores(scores, query_passages)

0.0322: 美肌を保つ保湿ケアと季節ごとのスキン管理術
スキンケアについては、水分補給が肝要だと気付きました。化粧水をいかに利用して保湿に繋げるかで、肌の状態のみならずメイクのノリも違ってきますから、積極的に化粧水を用いるようにしたいですね。お肌の水気が揮発して毛穴が乾燥してしまうと、毛穴にトラブルが齎される元になると考えられてしまうから、秋と冬は、いつも以上のケアが要されることになります。全力で乾燥肌を治したいなら、メイキャップは回避して、3時間経過する度に保湿だけのスキンケアを遂行することが、ベスト対策だそうです。ただし、実際のところは困難だと思えます。日々忙しい状態なので、なかなか睡眠時間が取れていないと考えていらっしゃる人もいるのではないでしょうか?とは言っても美白をお望みなら、睡眠時間を確保するようすることが大切だといえます。「日焼けする場所にいた!」と後悔している人も心配ご無用です。けれども、然るべきスキンケアを頑張ることが大切です。でもそれより先に、保湿をすることが最重要です。ほうれい線もしくはしわは、年齢を隠せないものです。「現在の年齢と比べて若く見えるか見えないか」というのは、このほうれい線としわの有無でジャッジが下されていると言われています。入浴した後、幾分時間が経ってからのスキンケアよりも、お肌の表面に水分が残っている入浴後すぐの方が、保湿効果は上がると言われています。一般的に、シミは新陳代謝のため皮膚表面まで持ち上げられる形になって、その内剥がれて見えなくなるのですが、メラニン色素が大量に作られることになると、皮膚に留まることとなりシミになってしまうのです。私達は多様なコスメ関連商品と美容関連情報に囲まれながら、一年を通して必死になってスキンケアに勤しんでいるわけです。ところが、そのやり方が正しくなければ、非常に高確率で乾燥肌になることも理解しておく必要があります。近頃の若者というのは、ストレスを溜め込むことが多いので、ニキビ向けの洗顔フォームや化粧水で手入れしているのに、全然快方に向かわないという状態なら、ストレスを抱え込んでいることが原因ではないかと思います。肌荒れを治したいのなら、普段より適切な暮らしを実践することが肝要になってきます。そんな中でも食生活を良化することで、体全体から肌荒れを改善し、美肌を作ることが何より理に適っている

98